In [1]:
import torch

# 张量计算图的数据结构

## DAG图存储   （有向无环图）

可以类比为哈夫曼树
这里的张量分为两个部分： 叶子张量 和 非叶子张量

- 叶子张量表示用户自己创建的张量   -->  叶子张量存储梯度，也就是日常神经网络的参数亦即内参
- 非叶子张量表示张量运算时候的中间结果   -->  神经网络的归一化层、隐藏层都是非叶子节点

## 算法

1. 创建计算图

- 创建一些张量表示叶子节点  <AccumulateGrad object>   --->  创建 a,b,c,d 四个张量 
- 通过叶子节点的运算操作创建非叶子节点
    - 对于一些多元运算操作，例如 y = a*b+c  系统会默认创建一个临时节点来存储 tmp = a*b 然后再用y节点来存储tmp和c

2. 反向传播

- 设置 path表，标记节点是否访问过。
    
- 每个非叶子节点，本身存储着关于子结点的函数表达外加子节点的列表。列表信息包括，子节点类型和索引
    
- 往上深度优先搜索。
    - 叶子节点： 
        - 为None: 设置grad属性
        - 不为None: 累加grad
    - 非叶子节点：将梯度计算的中间结果累乘并继续往上传播
    
    

In [72]:
# y = x^2

x = torch.arange(6).float()
x.requires_grad_(True)
y = x @ x
x

tensor([0., 1., 2., 3., 4., 5.], requires_grad=True)

In [73]:
y.backward()
x.grad

tensor([ 0.,  2.,  4.,  6.,  8., 10.])

In [74]:
x.grad.zero_()
y = x.sum()
y.backward()    # 这里的反向传播表示的是矩阵偏导
x.grad

tensor([1., 1., 1., 1., 1., 1.])

In [75]:
# 非标量反向传播
x.grad.zero_()
y = x * x
# y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([ 0.,  2.,  4.,  6.,  8., 10.])

In [112]:
# detack 张量分离，将分离出来的张量当成一个常数，然后求偏导
x.grad.zero_()
a = torch.arange(10,16).float()
b = torch.arange(20,26).float()
a.requires_grad_(True)
b.requires_grad_(True)
tmp = torch.stack([a,b],dim=0)
y = x*tmp
u = y    # python复制操作是浅拷贝，用于节省内存
z = u*x
z.sum().backward()
# x.grad
print(y.grad_fn.next_functions)

((<AccumulateGrad object at 0x7f969b5eab80>, 0), (<StackBackward0 object at 0x7f969c906d30>, 0))
